# 🛍️ 电商产品评分预测与推荐系统

> **项目目标**: 基于Amazon产品数据，预测产品评分并识别高评分产品

---

## 📋 项目概览

### 业务背景
电商平台需要:
- **预测新产品的评分** → 回归任务 (1.0-5.0)
- **识别高质量产品** → 分类任务 (评分 >= 4.0)

### 数据集
- **来源**: Kaggle Amazon Sales Dataset
- **规模**: 1,465条产品记录
- **特征**: 16个字段（价格、类别、评论、评分等）

### 技术栈
- **回归模型**: Linear Regression, Ridge, Lasso, Random Forest, XGBoost
- **分类模型**: Logistic Regression, SVM, Random Forest, XGBoost
- **特征工程**: TF-IDF文本特征、价格分桶、类别编码

---

## 🎯 学习目标

通过本项目，你将学会:
1. ✅ 加载并探索真实电商数据
2. ✅ 处理文本特征（TF-IDF）
3. ✅ 构建回归和分类模型
4. ✅ 对比多种算法性能
5. ✅ 提取业务洞察

---

## 📊 Notebook结构

本Notebook包含10个关键步骤，每个步骤都有清晰的TODO引导:

1. **环境准备与数据加载**
2. **数据概览**
3. **评分分布分析**
4. **价格与评分关系**
5. **类别分析**
6. **数据预处理**
7. **特征工程**
8. **回归模型训练与评估**
9. **分类模型训练与评估**
10. **模型对比与业务洞察**

---

**💡 使用说明**:
- 🔴 `# TODO:` 标记需要你填充的代码
- 💡 `提示:` 给出实现建议
- 📊 `预期结果:` 说明输出格式
- 🤔 `思考题:` 引导深入思考

---
# Step 1: 环境准备与数据加载

## 1.1 导入必要的库

In [ ]:
# TODO 1.1: 导入数据处理库
# 💡 提示: 需要 pandas, numpy

import pandas as pd
import numpy as np

# TODO 1.2: 导入可视化库
# 💡 提示: 需要 matplotlib, seaborn

import matplotlib.pyplot as plt
import seaborn as sns

# TODO 1.3: 导入机器学习库
# 💡 提示: sklearn的模型选择、预处理、评估模块

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# TODO 1.4: 导入回归模型
# 💡 提示: LinearRegression, Ridge, Lasso, RandomForestRegressor, XGBRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor  # 如果安装了xgboost

# TODO 1.5: 导入分类模型
# 💡 提示: LogisticRegression, SVC, RandomForestClassifier, XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier  # 如果安装了xgboost

# TODO 1.6: 导入评估指标
# 💡 提示: 回归指标(r2, MSE, MAE)，分类指标(accuracy, precision, recall, f1, roc_auc)

from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 设置可视化风格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

# 忽略警告
import warnings
warnings.filterwarnings('ignore')

print("✅ 环境准备完成!")

## 1.2 配置参数

In [ ]:
# TODO 1.7: 设置项目参数

# 路径配置
DATA_PATH = 'data/raw/amazon.csv'  # TODO: 确保数据文件路径正确

# 任务配置
TARGET_REGRESSION = 'rating'  # 回归目标：预测评分
TARGET_CLASSIFICATION = 'high_rating'  # 分类目标：预测高/低评分
HIGH_RATING_THRESHOLD = 4.0  # 高评分阈值

# 模型训练配置
TEST_SIZE = 0.2  # 测试集比例
CV_FOLDS = 5  # 交叉验证折数

# 特征工程配置
TFIDF_MAX_FEATURES = 100  # TF-IDF最大特征数
PRICE_BINS = [0, 500, 1000, 2000, 5000, 100000]  # 价格分桶

print("📋 参数配置完成")
print(f"   - 高评分阈值: {HIGH_RATING_THRESHOLD}")
print(f"   - 测试集比例: {TEST_SIZE}")
print(f"   - 交叉验证折数: {CV_FOLDS}")

## 1.3 加载数据

### 数据集列说明

Amazon销售数据集包含以下16列:

| 列名 | 说明 | 类型 |
|------|------|------|
| `product_id` | 产品唯一标识 | 字符串 |
| `product_name` | 产品名称 | 字符串 |
| `category` | 产品类别 | 字符串 |
| `discounted_price` | 折扣价格 | 数值 |
| `actual_price` | 原价 | 数值 |
| `discount_percentage` | 折扣百分比 | 数值 |
| `rating` | 产品评分 (1.0-5.0) | 数值 |
| `rating_count` | 评分数量 | 整数 |
| `about_product` | 产品描述 | 文本 |
| `user_id` | 用户ID | 字符串 |
| `user_name` | 用户名 | 字符串 |
| `review_id` | 评论ID | 字符串 |
| `review_title` | 评论标题 | 文本 |
| `review_content` | 评论内容 | 文本 |
| `img_link` | 图片链接 | 字符串 |
| `product_link` | 产品链接 | 字符串 |

In [ ]:
# TODO 1.8: 加载数据
# 💡 提示: 使用 pd.read_csv() 加载CSV文件
# 📊 预期结果: DataFrame with 1465 rows × 16 columns

df = None  # TODO: 在这里加载数据
# df = pd.read_csv(DATA_PATH)

# TODO 1.9: 显示数据基本信息
# 💡 提示: 使用 .shape, .head(), .info()

print("📊 数据集基本信息:")
# print(f"   - 数据形状: {df.shape}")
# print(f"   - 列名: {list(df.columns)}")

# TODO 1.10: 显示前5行数据
# df.head()

🤔 **思考题 1.1**:
- 数据集有多少行和列？
- 是否有缺失值？
- 数据类型是否正确（price应为数值型，category应为字符串型）？

---
# Step 2: 数据概览

## 2.1 数据结构检查

In [ ]:
# TODO 2.1: 查看数据类型和缺失值
# 💡 提示: 使用 df.info()
# 📊 预期结果: 显示每列的数据类型和非空值数量

# TODO: 在这里实现

## 2.2 数值特征统计

In [ ]:
# TODO 2.2: 查看数值型特征的统计信息
# 💡 提示: 使用 df.describe()
# 📊 预期结果: count, mean, std, min, 25%, 50%, 75%, max

# TODO: 在这里实现

# 关注以下列:
# - rating: 评分分布
# - discounted_price: 价格分布
# - discount_percentage: 折扣分布
# - rating_count: 评分数量分布

## 2.3 类别特征分布

In [ ]:
# TODO 2.3: 查看类别特征的唯一值数量
# 💡 提示: 使用 df['category'].value_counts()
# 📊 预期结果: 显示各类别的产品数量

print("📊 类别分布:")
# TODO: 在这里实现

# TODO 2.4: 统计有多少个不同的类别
# num_categories = df['category'].nunique()
# print(f"\n总共有 {num_categories} 个不同的类别")

🤔 **思考题 2.1**:
- 哪个类别的产品最多？
- 是否存在样本不平衡问题？
- 是否需要对小样本类别进行处理？

---
# Step 3: 评分分布分析

## 3.1 评分统计量

In [ ]:
# TODO 3.1: 计算评分的统计量
# 💡 提示: 使用 df['rating'].describe() 和自定义统计
# 📊 预期结果: mean, median, std, min, max, quartiles

print("📊 评分统计量:")
# TODO: 在这里实现

# 提示: 可以计算以下指标
# rating_mean = df['rating'].mean()
# rating_median = df['rating'].median()
# rating_std = df['rating'].std()
# rating_min = df['rating'].min()
# rating_max = df['rating'].max()

## 3.2 评分分布可视化

In [ ]:
# TODO 3.2: 绘制评分分布直方图
# 💡 提示: 使用 plt.hist() 或 sns.histplot()
# 📊 预期结果: 带KDE曲线的直方图，标注均值和中位数线

# fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# TODO: 在这里实现 4 个子图
# 子图1: 直方图 + KDE
# 子图2: 评分计数柱状图
# 子图3: 箱线图
# 子图4: 饼图（高/低评分占比）

# plt.tight_layout()
# plt.show()

## 3.3 创建分类目标变量

In [ ]:
# TODO 3.3: 创建高评分标签（二分类目标）
# 💡 提示: rating >= 4.0 为高评分 (1)，否则为低评分 (0)
# 📊 预期结果: 新增 'high_rating' 列，值为 0 或 1

# df['high_rating'] = (df['rating'] >= HIGH_RATING_THRESHOLD).astype(int)

# TODO 3.4: 统计高评分产品占比
# high_rating_ratio = df['high_rating'].mean()
# print(f"\n📊 高评分产品占比: {high_rating_ratio:.2%}")
# print(f"   - 高评分产品 (≥{HIGH_RATING_THRESHOLD}): {df['high_rating'].sum()} 个")
# print(f"   - 低评分产品 (<{HIGH_RATING_THRESHOLD}): {(1-df['high_rating']).sum()} 个")

🤔 **思考题 3.1**:
- 评分分布是否偏态？偏向高分还是低分？
- 高评分产品占比是否平衡？是否需要处理类别不平衡？
- 如果大部分产品都是高评分，这对模型训练有什么影响？

---
# Step 4: 价格与评分关系分析

## 4.1 价格统计

In [ ]:
# TODO 4.1: 分析价格分布
# 💡 提示: 分析 discounted_price 和 actual_price
# 📊 预期结果: 价格的统计量和分布特征

print("💰 价格统计:")
# TODO: 在这里实现

## 4.2 价格与评分的关系

In [ ]:
# TODO 4.2: 绘制价格与评分的散点图
# 💡 提示: 使用 plt.scatter() 或 sns.scatterplot()
# 📊 预期结果: 散点图 + 趋势线，观察价格和评分的相关性

# fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# TODO: 在这里实现 2 个子图
# 子图1: discounted_price vs rating
# 子图2: discount_percentage vs rating

# plt.tight_layout()
# plt.show()

## 4.3 相关性分析

In [ ]:
# TODO 4.3: 计算数值特征之间的相关性
# 💡 提示: 使用 df[['rating', 'discounted_price', 'discount_percentage', 'rating_count']].corr()
# 📊 预期结果: 相关性矩阵和热力图

# TODO: 在这里实现
# 提示: 使用 sns.heatmap() 绘制相关性热力图

🤔 **思考题 4.1**:
- 价格和评分之间有相关性吗？是正相关还是负相关？
- 折扣力度是否影响评分？
- 评分数量（rating_count）和评分（rating）有关系吗？

---
# Step 5: 类别分析

## 5.1 各类别的评分分布

In [ ]:
# TODO 5.1: 分析不同类别的评分分布
# 💡 提示: 使用 df.groupby('category')['rating'].agg(['mean', 'median', 'count'])
# 📊 预期结果: 显示各类别的平均评分、中位数评分和产品数量

print("📊 各类别评分统计:")
# TODO: 在这里实现

## 5.2 类别评分可视化

In [ ]:
# TODO 5.2: 绘制各类别的评分箱线图
# 💡 提示: 使用 sns.boxplot()
# 📊 预期结果: 箱线图，展示各类别的评分分布和离群点

# 如果类别太多，可以只选择Top 10类别
# top_categories = df['category'].value_counts().head(10).index
# df_top = df[df['category'].isin(top_categories)]

# TODO: 在这里实现

🤔 **思考题 5.1**:
- 哪个类别的平均评分最高？哪个最低？
- 类别之间的评分差异大吗？
- 这种差异对建模有什么启示？

---
# Step 6: 数据预处理

## 6.1 处理缺失值

In [ ]:
# TODO 6.1: 检查缺失值
# 💡 提示: 使用 df.isnull().sum()
# 📊 预期结果: 显示每列的缺失值数量

print("🔍 缺失值检查:")
# TODO: 在这里实现

# TODO 6.2: 处理缺失值
# 💡 提示: 根据实际情况选择策略
#   - 文本字段: 填充为空字符串 ''
#   - 数值字段: 填充为中位数或均值
#   - 评分字段: 如果缺失，可能需要删除该行

# TODO: 在这里实现

## 6.2 处理异常值

In [ ]:
# TODO 6.3: 检测异常值
# 💡 提示: 使用箱线图规则 (IQR方法) 或 Z-score
# 📊 预期结果: 识别价格、评分等字段的异常值

# TODO: 在这里实现

# 示例: 检测价格异常值
# Q1 = df['discounted_price'].quantile(0.25)
# Q3 = df['discounted_price'].quantile(0.75)
# IQR = Q3 - Q1
# outliers = df[(df['discounted_price'] < Q1 - 1.5*IQR) | (df['discounted_price'] > Q3 + 1.5*IQR)]

## 6.3 数据类型转换

In [ ]:
# TODO 6.4: 转换数据类型
# 💡 提示: 确保价格列为float，category为字符串，rating_count为int
# 📊 预期结果: 数据类型正确，便于后续处理

# TODO: 在这里实现

# 示例:
# df['discounted_price'] = df['discounted_price'].astype(float)
# df['rating_count'] = df['rating_count'].astype(int)

🤔 **思考题 6.1**:
- 对于缺失值，删除和填充各有什么优缺点？
- 异常值一定要删除吗？什么情况下可以保留？
- 数据清洗对后续建模有多重要？

---
# Step 7: 特征工程

## 7.1 价格特征工程

In [ ]:
# TODO 7.1: 创建价格相关特征
# 💡 提示: 可以创建以下特征
#   - log_price: 价格对数（处理偏态分布）
#   - price_bin: 价格分桶（低/中/高价位）
#   - discount_amount: 折扣金额 = actual_price - discounted_price

# TODO: 在这里实现

# 示例: 创建对数价格
# df['log_price'] = np.log1p(df['discounted_price'])

# 示例: 价格分桶
# df['price_bin'] = pd.cut(df['discounted_price'], bins=PRICE_BINS, labels=['very_low', 'low', 'medium', 'high', 'very_high'])

## 7.2 文本特征提取 (TF-IDF)

In [ ]:
# TODO 7.2: 从评论文本中提取TF-IDF特征
# 💡 提示: 使用 TfidfVectorizer
# 📊 预期结果: 从 review_content 或 about_product 提取文本特征

# TODO: 在这里实现

# 示例:
# tfidf = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, min_df=2, max_df=0.8, stop_words='english')
# tfidf_features = tfidf.fit_transform(df['review_content'].fillna(''))
# tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=[f'tfidf_{i}' for i in range(TFIDF_MAX_FEATURES)])
# df = pd.concat([df, tfidf_df], axis=1)

## 7.3 类别特征编码

In [ ]:
# TODO 7.3: 对类别特征进行编码
# 💡 提示: 使用 One-Hot Encoding 或 Label Encoding
# 📊 预期结果: 将 category 转换为数值特征

# TODO: 在这里实现

# 方法1: One-Hot Encoding (适合类别少的情况)
# category_dummies = pd.get_dummies(df['category'], prefix='category')
# df = pd.concat([df, category_dummies], axis=1)

# 方法2: Label Encoding (适合类别多的情况)
# le = LabelEncoder()
# df['category_encoded'] = le.fit_transform(df['category'])

## 7.4 交互特征

In [ ]:
# TODO 7.4: 创建交互特征
# 💡 提示: 可以创建以下交互特征
#   - price_per_rating: discounted_price / (rating_count + 1)
#   - discount_rating_interaction: discount_percentage * rating_count

# TODO: 在这里实现

## 7.5 特征选择

In [ ]:
# TODO 7.5: 选择最终用于建模的特征
# 💡 提示: 排除无关列（product_id, product_name, img_link等）
# 📊 预期结果: 定义 feature_columns 列表

# TODO: 在这里实现

# 示例:
# feature_columns = ['discounted_price', 'discount_percentage', 'rating_count', 'log_price'] + \
#                   [col for col in df.columns if col.startswith('tfidf_')] + \
#                   [col for col in df.columns if col.startswith('category_')]

# print(f"\n✅ 选择了 {len(feature_columns)} 个特征用于建模")

🤔 **思考题 7.1**:
- 为什么要对价格取对数？
- TF-IDF如何将文本转换为数值特征？
- One-Hot Encoding和Label Encoding的区别是什么？各适用于什么场景？

---
# Step 8: 回归模型训练与评估

## 8.1 准备数据

In [ ]:
# TODO 8.1: 划分训练集和测试集（回归任务）
# 💡 提示: 使用 train_test_split
# 📊 预期结果: X_train_reg, X_test_reg, y_train_reg, y_test_reg

# TODO: 在这里实现

# X = df[feature_columns]
# y_reg = df[TARGET_REGRESSION]  # 'rating'

# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
#     X, y_reg, test_size=TEST_SIZE, random_state=RANDOM_STATE
# )

# print(f"训练集大小: {X_train_reg.shape}")
# print(f"测试集大小: {X_test_reg.shape}")

## 8.2 特征标准化

In [ ]:
# TODO 8.2: 标准化特征
# 💡 提示: 使用 StandardScaler
# 📊 预期结果: 标准化后的训练集和测试集

# TODO: 在这里实现

# scaler = StandardScaler()
# X_train_reg_scaled = scaler.fit_transform(X_train_reg)
# X_test_reg_scaled = scaler.transform(X_test_reg)

## 8.3 训练多个回归模型

In [ ]:
# TODO 8.3: 训练并评估多个回归模型
# 💡 提示: 训练 Linear Regression, Ridge, Lasso, Random Forest, XGBoost
# 📊 预期结果: 每个模型的 R², RMSE, MAE

# TODO: 在这里实现

# 示例框架:
# regression_results = {}

# models = {
#     'Linear Regression': LinearRegression(),
#     'Ridge': Ridge(alpha=1.0),
#     'Lasso': Lasso(alpha=1.0),
#     'Random Forest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE),
#     # 'XGBoost': XGBRegressor(n_estimators=100, random_state=RANDOM_STATE)
# }

# for name, model in models.items():
#     # TODO: 训练模型
#     # model.fit(X_train_reg_scaled, y_train_reg)
#     
#     # TODO: 预测
#     # y_pred = model.predict(X_test_reg_scaled)
#     
#     # TODO: 评估
#     # r2 = r2_score(y_test_reg, y_pred)
#     # rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred))
#     # mae = mean_absolute_error(y_test_reg, y_pred)
#     
#     # regression_results[name] = {'R²': r2, 'RMSE': rmse, 'MAE': mae}
#     pass

## 8.4 模型对比

In [ ]:
# TODO 8.4: 对比各模型的性能
# 💡 提示: 将结果整理为DataFrame并可视化
# 📊 预期结果: 柱状图对比各模型的R²和RMSE

# TODO: 在这里实现

# results_df = pd.DataFrame(regression_results).T
# print("\n📊 回归模型性能对比:")
# print(results_df)

# TODO: 绘制对比图

🤔 **思考题 8.1**:
- 哪个模型的R²最高？为什么？
- RMSE和MAE的区别是什么？
- 线性模型和树模型哪个更适合这个任务？

---
# Step 9: 分类模型训练与评估

## 9.1 准备数据

In [ ]:
# TODO 9.1: 划分训练集和测试集（分类任务）
# 💡 提示: 使用 train_test_split
# 📊 预期结果: X_train_clf, X_test_clf, y_train_clf, y_test_clf

# TODO: 在这里实现

# y_clf = df[TARGET_CLASSIFICATION]  # 'high_rating'

# X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
#     X, y_clf, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_clf
# )

# 标准化
# X_train_clf_scaled = scaler.fit_transform(X_train_clf)
# X_test_clf_scaled = scaler.transform(X_test_clf)

## 9.2 训练多个分类模型

In [ ]:
# TODO 9.2: 训练并评估多个分类模型
# 💡 提示: 训练 Logistic Regression, SVM, Random Forest, XGBoost
# 📊 预期结果: 每个模型的 Accuracy, Precision, Recall, F1, ROC-AUC

# TODO: 在这里实现

# classification_results = {}

# models = {
#     'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
#     'SVM': SVC(random_state=RANDOM_STATE, probability=True),
#     'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
#     # 'XGBoost': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE)
# }

# for name, model in models.items():
#     # TODO: 训练、预测、评估
#     pass

## 9.3 混淆矩阵

In [ ]:
# TODO 9.3: 绘制最佳模型的混淆矩阵
# 💡 提示: 使用 confusion_matrix 和 sns.heatmap
# 📊 预期结果: 混淆矩阵热力图

# TODO: 在这里实现

## 9.4 ROC曲线

In [ ]:
# TODO 9.4: 绘制ROC曲线
# 💡 提示: 使用 roc_curve 和 plt.plot
# 📊 预期结果: ROC曲线，标注AUC值

# TODO: 在这里实现

🤔 **思考题 9.1**:
- Accuracy、Precision、Recall、F1的区别是什么？
- 什么情况下应该关注Precision？什么情况下关注Recall？
- ROC-AUC为什么是衡量分类器的好指标？

---
# Step 10: 模型对比与业务洞察

## 10.1 综合对比所有模型

In [ ]:
# TODO 10.1: 综合展示所有模型的性能
# 💡 提示: 将回归和分类结果整合，绘制对比图
# 📊 预期结果: 表格 + 可视化对比

# TODO: 在这里实现

## 10.2 特征重要性分析

In [ ]:
# TODO 10.2: 提取并可视化特征重要性
# 💡 提示: 使用Random Forest或XGBoost的feature_importances_
# 📊 预期结果: 柱状图展示Top 20重要特征

# TODO: 在这里实现

## 10.3 业务洞察与建议

### 📊 模型分析总结

**TODO 10.3: 根据分析结果，回答以下问题**

1. **哪些因素最影响产品评分？**
   - 💡 提示: 查看特征重要性排名
   - 你的答案: 

2. **价格、折扣与评分的关系？**
   - 💡 提示: 回顾Step 4的相关性分析
   - 你的答案: 

3. **不同类别产品的评分差异？**
   - 💡 提示: 回顾Step 5的类别分析
   - 你的答案: 

4. **文本特征的作用？**
   - 💡 提示: TF-IDF特征在模型中的重要性如何？
   - 你的答案: 

### 💼 业务建议

**TODO 10.4: 基于模型结果，提出业务建议**

1. **对于产品运营团队**:
   - 你的建议: 

2. **对于定价策略**:
   - 你的建议: 

3. **对于新产品上架**:
   - 你的建议: 

4. **对于用户评价管理**:
   - 你的建议: 

## 10.4 模型保存

In [ ]:
# TODO 10.5: 保存最佳模型
# 💡 提示: 使用 joblib.dump() 保存模型、scaler、tfidf等
# 📊 预期结果: 保存到 models/ 目录

# TODO: 在这里实现

# import joblib
# joblib.dump(best_regression_model, 'models/rating_regression_model.pkl')
# joblib.dump(best_classification_model, 'models/rating_classification_model.pkl')
# joblib.dump(scaler, 'models/scaler.pkl')
# joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')

print("✅ 模型保存完成!")

---
# 🎉 项目完成!

## 📝 学习总结

恭喜你完成了这个完整的电商评分预测项目！

### 你学到了:

1. ✅ **数据加载与探索** - 处理真实的电商数据集
2. ✅ **数据预处理** - 缺失值、异常值处理
3. ✅ **特征工程** - 价格特征、TF-IDF文本特征、类别编码
4. ✅ **回归建模** - 预测连续评分 (1.0-5.0)
5. ✅ **分类建模** - 预测高/低评分
6. ✅ **模型评估** - R², RMSE, Accuracy, ROC-AUC等指标
7. ✅ **模型对比** - 多种算法性能对比
8. ✅ **业务洞察** - 将技术结果转化为业务价值

### 下一步建议:

1. **优化模型**:
   - 尝试超参数调优 (GridSearchCV)
   - 尝试更多特征工程方法
   - 尝试模型融合 (Stacking)

2. **深入分析**:
   - 分析错误案例（评分预测差距大的产品）
   - 进行A/B测试验证模型效果
   - 添加时间维度分析（评分趋势）

3. **部署应用**:
   - 使用保存的模型进行实时预测
   - 构建简单的Web界面 (Streamlit/Flask)
   - 定期重训练模型

4. **扩展项目**:
   - 添加推荐系统功能
   - 进行情感分析（评论文本）
   - 构建用户画像

---

**🚀 继续学习，持续进步！**

如果有任何问题，请参考:
- 项目文档: `README.md`
- Python模块代码: `src/` 目录
- 快速入门: `QUICKSTART.md`